<a href="https://colab.research.google.com/github/yardenfren1996/B-LoRA/blob/main/B_LoRA_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installation

**1.1 Clone the Repository :**

In [1]:
# Clone the GitHub repository containing the code and resources needed for the project.
!git clone https://github.com/RongWenYin/improved_blora.git

# Change the current working directory to the newly cloned repository folder.
%cd improved_blora

fatal: destination path 'improved_blora' already exists and is not an empty directory.
/kaggle/working/improved_blora


**1.2 Install Dependencies :**

In [2]:
# Install all dependencies listed in the requirements.txt file to ensure the environment has the necessary packages.
!pip install -r requirements.txt

**1.3 Initial Parameters :**

In [3]:
# styleKey should match one of the folder names in the 'images' folder from the GitHub repository.
# Available options include: 'autumn', 'car', 'color_building', 'color_pen', 'dinosaur', 'leaf', 'watercolor'.
styleKey = 'grape'

# promptKey sets the specific prompt for generating images or applying styles.
# You can keep the default value "[s90]" as it works universally with this script.
promptKey = "[s90]"

# output_dir defines the directory where the processed or generated outputs will be saved.
# It uses the styleKey to create a unique folder name for each style, like 'outputs_grape'.
output_dir = f'outputs_{styleKey}'


**1.4 suppress the FutureWarning**

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 2. Train the LoRA within specific blocks only 

**(optional step; skip to use the pre-trained LoRA from Hugging Face in Step 3)**

In [ ]:
# Launch the training script for fine-tuning the Stable Diffusion XL model with b-Lora.
# This uses the Accelerate library to distribute the training across available resources.

# Using the specified cust_block_list, only the LoRA layers within attention blocks W1 through W6 will be trained.
!accelerate launch train_dreambooth_b-lora_sdxl.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
    --instance_data_dir="./images/{styleKey}" \
    --output_dir="{output_dir}" \
    --instance_prompt="{promptKey}" \
    --resolution=1024 \
    --rank=64 \
    --train_batch_size=1 \
    --learning_rate=5e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps=1000 \
    --checkpointing_steps=1000 \
    --seed="0" \
    --gradient_checkpointing \
    --use_8bit_adam \
    --mixed_precision="fp16" \
    --cust_block_list="down_blocks.2.attentions.0 down_blocks.2.attentions.1 mid_block.attentions.0 up_blocks.0.attentions.0 up_blocks.0.attentions.1 up_blocks.0.attentions.2"


# 3. Inference : 

**3.1 Import utility functions :**

In [9]:
# Import all utility functions and classes from the GitHub repository's utils module
from utils import *

outputs_grape/pytorch_lora_weights.safetensors
{'content': ['unet.up_blocks.0.attentions.0'], 'style': ['unet.up_blocks.0.attentions.1', 'unet.up_blocks.0.attentions.2'], 'W1': ['unet.down_blocks.2.attentions.0'], 'W2': ['unet.down_blocks.2.attentions.1'], 'W3': ['unet.mid_block.attentions.0'], 'W4': ['unet.up_blocks.0.attentions.0'], 'W5': ['unet.up_blocks.0.attentions.1'], 'W6': ['unet.up_blocks.0.attentions.2']}


**3.2 Download or specify the lora model :**

In [ ]:
# Define the file path for the trained LoRA weights, specifying the output directory and checkpoint location
style_B_LoRA_path=hf_hub_download(repo_id="ElaineYin/loras", filename="grape_all_1000.safetensors", local_dir="loras")

# Uncomment the line below to use the LoRA model created in Step 2, instead of the Hugging Face LoRA files:
# style_B_LoRA_path = f'{output_dir}/pytorch_lora_weights.safetensors'

**3.3 Specify the object to be generated   :**

In [ ]:
# List of object names to be used as prompts in image generation
objectNames = ["girl", "cat", "apple", "dog", "fish"]

**3.4 Generate Images using the SDXL Pipeline with Improved B-LoRA :**

In [ ]:
# The inferenceImages function generates images based on a list of object names,
# leveraging the LoRA model to apply specific styles across all combinations of attention blocks from W1 to W6.
inferenceImages(objectNames)

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Applying B-LoRA to blocks: ['unet.down_blocks.2.attentions.0']
Generating images for [s90] style | W1 layer | 1000 steps


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Applying B-LoRA to blocks: ['unet.up_blocks.0.attentions.1']
Generating images for [s90] style | W5 layer | 1000 steps


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
!zip -j lora_civitai.zip ./*.png

In [ ]:
!rm ./*.png